In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

import pandas_profiling
from pandas_profiling import ProfileReport

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows", None)

In [9]:
df=pd.read_csv('PBL 5 recommendation data.csv',encoding='iso-8859-1')
df_all=pd.read_csv('PBL 5 recommendation data.csv',encoding='iso-8859-1')

In [10]:
# profile = ProfileReport(df, title="Pandas Profiling Report")
# profile.to_file("PBL5recommendationdata_EDA.html")

In [11]:
df.sample()

,Customers.id,Customers.fname,Customers.lname,Customers.company,Customers.create_date,Customers.status,Customers.mailing,Customers.reminders,Customers.tax_exempt,Customers.account_id,Customers.sales_rep,Customers.rewards,Customers.profile_id,Customers.last_modified,Customers.customer_type,Orders.id,Orders.customer_id,Orders.fname,Orders.lname,Orders.company,Orders.order_number,Orders.reorder_id,Orders.external_source,Orders.external_id,Orders.currency,Orders.sales_rep,Orders.subtotal,Orders.tax,Orders.shipping,Orders.coupon_id,Orders.coupon_amount,Orders.gift_id,Orders.gift_amount,Orders.fee_name,Orders.fee_amount,Orders.discount_name,Orders.discount_amount,Orders.total,Orders.balance_due,Orders.shipping_carrier,Orders.shipping_method,Orders.shipping_trans,Orders.shipping_flags,Orders.weight,Orders.tracking,Orders.payment_status,Orders.payment_date,Orders.payment_user,Orders.payment_type,Orders.payment_method,Orders.payment_amount,Orders.purchase_order,Orders.payment_id,Orders.payment_code,Orders.payment_ref,Orders.status,Orders.placed_date,Orders.updated_date,Orders.shipped_date,Orders.comments,Orders.notes,Orders.registry_id,Orders.gift_message,Orders.website,Orders.mailing,Orders.flags,Orders.partial_ship,Orders.customer_type,Order_Items.id,Order_Items.parent,Order_Items.product_id,Order_Items.product_name,Order_Items.attributes,Order_Items.attribute_names,Order_Items.attribute_prices,Order_Items.qty,Order_Items.price,Order_Items.cost,Order_Items.registry_item,Order_Items.related_id,Order_Items.reorder_frequency,Order_Items.account_id,Order_Items.flags,Products.id,Products.status,Products.product_type,Products.template,Products.vendor,Products.import_id,Products.name,Products.display_name,Products.menu_name,Products.list_price,Products.price,Products.sale_price,Products.cost,Products.flags,Products.left_flag,Products.right_flag,Products.last_modified,Products.taxable,Products.shopping_gtin,Products.shopping_brand,Products.shopping_mpn,Products.shopping_gender,Products.shopping_color,Products.shopping_age,Products.shopping_flags,Products.amazon_asin,Products.amazon_type,Products.amazon_item_type,Products.amazon_price,Products.google_shopping_id,Products.google_shopping_type,Products.google_shopping_cat,Products.google_adwords,Products.shopping_cat,Products.shopping_type,Products.pricegrabber_cat,Products.shopzilla_cat,Products.thefind_cat,Products.quickbooks_id,Products.qb_edit_sequence,Products.price_break_type,Products.price_breaks,Products.short_description,Products.long_description,Products.websites,Products.video,Products.audio,Products.seo_title,Products.seo_description,Products.seo_keywords,Products.seo_header,Products.seo_footer,Products.seo_url,Products.seo_category,Products.unit,Products.packaging,Products.display_packaging,Products.multiple,Products.length,Products.width,Products.height,Products.rx,Products.latex,Products.upc,Products.msds_link,Products.msds_label,Products.lit_link,Products.lit_label,Products.hcpcs,Products.case_qty,Products.markup,Products.override_markup,Products.notes,Products.import_flags,Products.map_price,Products.features_title,Products.warranty,Products.hygienic,Products.default_quantity,Products.user_size,Products.assembly,Products.installation,Products.shipping_length,Products.shipping_width,Products.shipping_height,Products.shipping_weight,Products.handling_time,Products.rotation_link,Products.google_shopping_label,Products.product_option,Products.size,Products.material,Products.arm_style,Products.leg_style,Products.seat_size,Products.family_id,Products.saved_status,Products.freight_cost
3554,3096,Davita,Nowland,NaN,1459961077,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1459961077,0.0,15210,3096,Davita,Nowland,NaN,15210,NaN,NaN,NaN,USD,NaN,184.4,0.0,9.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,194.35,NaN,fedex,11|Ground,NaN,NaN,NaN,4.8383E+14,3.0,1.459961e+09,NaN,authorize.net,NaN,194.35,NaN,8139332286,05028B,NaN,1,1459961310,1.459986e+09,1.459983e+09,This is for Don.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17778,

In [12]:
df.shape

(4194, 181)

In [13]:
df.isnull().sum()

Customers.id                         0
Customers.fname                      0
Customers.lname                      0
Customers.company                 3467
Customers.create_date                0
Customers.status                  4093
Customers.mailing                 2080
Customers.reminders               4194
Customers.tax_exempt              4189
Customers.account_id              4191
Customers.sales_rep               4194
Customers.rewards                 4194
Customers.profile_id              4194
Customers.last_modified              0
Customers.customer_type            423
Orders.id                            0
Orders.customer_id                   0
Orders.fname                         0
Orders.lname                         0
Orders.company                    3466
Orders.order_number                  0
Orders.reorder_id                 4112
Orders.external_source            3829
Orders.external_id                4189
Orders.currency                      0
Orders.sales_rep         

In [16]:
# Delete where is the empty columns
df=df.dropna(axis=1,how="any")
# Some columns name that check
df["Customers.company"]=df_all["Customers.company"]
df["Products.shopping_brand"]=df_all["Products.shopping_brand"]
df["Orders.payment_method"]=df_all["Orders.payment_method"]
df["Full_Name"]=df["Customers.fname"]+" "+df["Customers.lname"]
df["Orders.shipping_method"]=df_all["Orders.shipping_method"]

df.sample(5)

,Customers.id,Customers.fname,Customers.lname,Customers.create_date,Customers.last_modified,Orders.id,Orders.customer_id,Orders.fname,Orders.lname,Orders.order_number,Orders.currency,Orders.subtotal,Orders.total,Orders.status,Orders.placed_date,Order_Items.id,Order_Items.parent,Order_Items.product_name,Order_Items.qty,Order_Items.price,Customers.company,Products.shopping_brand,Orders.payment_method,Full_Name,Orders.shipping_method
3018,2587,Richard,Kimble,1456106094,1456953653,13044,2587,Richard,Kimble,13044,USD,57.19,61.42,1,1456106344,15449,13044,"Drive Medical Adjustable, Non-Tilt Overbed Table",1,57.19,NaN,Drive Medical,PayPal,Richard Kimble,0|Standard Shipping
1052,866,FEB,KNODERER,1427326221,1437763123,3890,866,FEB,KNODERER,3890,USD,80.72,86.63,1,1427326310,5435,3890,Gel Foam Pressure Redistribution Cushions,1,30.86,NaN,MEDLINE,NaN,FEB KNODERER,11|Ground
2175,1805,Eva,Lopez,1447953049,1456954471,8581,1805,Eva,Lopez,8581,USD,633.02,633.02,1,1447953050,10668,8581,"First Class School Chair Anti Tippers, Small, ...",2,29.69,Company173,Wenzelite,NaN,Eva Lopez,NaN
1337,1089,Jennifer,Jones,1432686729,1437559843,4375,1089,Jennifer,Jones,4375,USD,38.08,48.03,1,1432687426,5974,4375,"RPO Seat Pads, Beige",1,38.08,NaN,MEDLINE,NaN,Jennifer Jones,11|Ground
2397,2019,Van,Tran,1450910682,1456954564,10601,2019,Van,Tran,10601,USD,86.01,95.96,1,1450910687,12761,10601,Single Ply Polypropylene Bouffant Caps,1,12.94,Company205,MEDLINE,NaN,Van Tran,11|Ground


In [17]:
df.isnull().sum()

Customers.id                   0
Customers.fname                0
Customers.lname                0
Customers.create_date          0
Customers.last_modified        0
Orders.id                      0
Orders.customer_id             0
Orders.fname                   0
Orders.lname                   0
Orders.order_number            0
Orders.currency                0
Orders.subtotal                0
Orders.total                   0
Orders.status                  0
Orders.placed_date             0
Order_Items.id                 0
Order_Items.parent             0
Order_Items.product_name       0
Order_Items.qty                0
Order_Items.price              0
Customers.company           3467
Products.shopping_brand      162
Orders.payment_method       3472
Full_Name                      0
Orders.shipping_method       443
dtype: int64

In [15]:
# check the most populer field name 
def the_most_popular(column_name,ranking="total"): 
    
    grouped=df.groupby(column_name).agg({
        "Order_Items.qty":[np.size,np.sum,np.mean],
        "Order_Items.price":[np.mean]})
    grouped["Total_Amount"]=grouped["Order_Items.qty"]["sum"]*grouped["Order_Items.price"]["mean"]
    
    if ranking=="sum":
        return grouped.sort_values(("Order_Items.qty","sum"),ascending=False).head(5)
    else:
        return grouped.sort_values(("Total_Amount"),ascending=False).head(5)

In [18]:
the_most_popular("Order_Items.product_name")

Order_Items.qty       \
                                                              size  sum   
Order_Items.product_name                                                  
Disposable Emergency Blanket, Gray, Not Applicable              14  305   
Medline MoliCare Super Plus Disposable Adult Di...              53  164   
Medline Deluxe Aluminum Transport Chair With Ha...              46   47   
Sanitary Pads with Adhesive & Wings                              2  133   
2-Ply Tissue/Poly Professional Towels, White, N...               2  220   

                                                                \
                                                          mean   
Order_Items.product_name                                         
Disposable Emergency Blanket, Gray, Not Applicable   21.785714   
Medline MoliCare Super Plus Disposable Adult Di...    3.094340   
Medline Deluxe Aluminum Transport Chair With Ha...    1.021739   
Sanitary Pads with Adhesive & Wings                  66.500000   
2-Ply Tissue/Poly Professional Towels, White, N...  110.000000   

                                                   Order_Items.price  \
                                                                mean   
Order_Items.product_name                                               
Disposable Emergency Blanket, Gray, Not Applicable         34.610000   
Medline MoliCare Super Plus Disposable Adult Di...         54.760377   
Medline Deluxe Aluminum Transport Chair With Ha...        126.620870   
Sanitary Pads with Adhesive & Wings                        42.320000   
2-Ply Tissue/Poly Professional Towels, White, N...         25.270000   

                                                    Total_Amount  
                                                                  
Order_Items.product_name                                          
Disposable Emergency Blanket, Gray, Not Applicable  10556.050000  
Medline MoliCare Super Plus Disposable Adult Di...   8980.701887  
Medline Deluxe Aluminum Transport Chair With Ha...   5951.180870  
Sanitary Pads with Adhesive & Wings                  5628.560000  
2-Ply Tissue/Poly Professional Towels, White, N...   5559.400000

In [19]:
the_most_popular("Order_Items.product_name","sum")

Order_Items.qty       \
                                                              size  sum   
Order_Items.product_name                                                  
TruTemp                                                          1  500   
Disposable Emergency Blanket, Gray, Not Applicable              14  305   
2-Ply Tissue/Poly Professional Towels, White, N...               2  220   
Medline MoliCare Super Plus Disposable Adult Di...              53  164   
Sanitary Pads with Adhesive & Wings                              2  133   

                                                                \
                                                          mean   
Order_Items.product_name                                         
TruTemp                                             500.000000   
Disposable Emergency Blanket, Gray, Not Applicable   21.785714   
2-Ply Tissue/Poly Professional Towels, White, N...  110.000000   
Medline MoliCare Super Plus Disposable Adult Di...    3.094340   
Sanitary Pads with Adhesive & Wings                  66.500000   

                                                   Order_Items.price  \
                                                                mean   
Order_Items.product_name                                               
TruTemp                                                     2.990000   
Disposable Emergency Blanket, Gray, Not Applicable         34.610000   
2-Ply Tissue/Poly Professional Towels, White, N...         25.270000   
Medline MoliCare Super Plus Disposable Adult Di...         54.760377   
Sanitary Pads with Adhesive & Wings                        42.320000   

                                                    Total_Amount  
                                                                  
Order_Items.product_name                                          
TruTemp                                              1495.000000  
Disposable Emergency Blanket, Gray, Not Applicable  10556.050000  
2-Ply Tissue/Poly Professional Towels, White, N...   5559.400000  
Medline MoliCare Super Plus Disposable Adult Di...   8980.701887  
Sanitary Pads with Adhesive & Wings                  5628.560000

In [20]:
the_most_popular("Customers.company")

Order_Items.qty                 Order_Items.price  \
                             size  sum       mean              mean   
Customers.company                                                     
Company59                      15  381  25.400000         34.610000   
Company343                      2  133  66.500000         42.320000   
Company17                       2   27  13.500000        177.190000   
Company159                     11   47   4.272727         80.058182   
Company281                      4   43  10.750000         62.910000   

                   Total_Amount  
                                 
Customers.company                
Company59          13186.410000  
Company343          5628.560000  
Company17           4784.130000  
Company159          3762.734545  
Company281          2705.130000

In [21]:
the_most_popular("Customers.company","sum")

Order_Items.qty                 Order_Items.price  \
                             size  sum       mean              mean   
Customers.company                                                     
Company59                      15  381  25.400000         34.610000   
Company343                      2  133  66.500000         42.320000   
Company86                      11  105   9.545455         18.400000   
Company130                     17   71   4.176471         28.382353   
Company171                     12   47   3.916667         31.941667   

                   Total_Amount  
                                 
Customers.company                
Company59          13186.410000  
Company343          5628.560000  
Company86           1932.000000  
Company130          2015.147059  
Company171          1501.258333

In [22]:
the_most_popular("Order_Items.product_name","total")

Order_Items.qty       \
                                                              size  sum   
Order_Items.product_name                                                  
Disposable Emergency Blanket, Gray, Not Applicable              14  305   
Medline MoliCare Super Plus Disposable Adult Di...              53  164   
Medline Deluxe Aluminum Transport Chair With Ha...              46   47   
Sanitary Pads with Adhesive & Wings                              2  133   
2-Ply Tissue/Poly Professional Towels, White, N...               2  220   

                                                                \
                                                          mean   
Order_Items.product_name                                         
Disposable Emergency Blanket, Gray, Not Applicable   21.785714   
Medline MoliCare Super Plus Disposable Adult Di...    3.094340   
Medline Deluxe Aluminum Transport Chair With Ha...    1.021739   
Sanitary Pads with Adhesive & Wings                  66.500000   
2-Ply Tissue/Poly Professional Towels, White, N...  110.000000   

                                                   Order_Items.price  \
                                                                mean   
Order_Items.product_name                                               
Disposable Emergency Blanket, Gray, Not Applicable         34.610000   
Medline MoliCare Super Plus Disposable Adult Di...         54.760377   
Medline Deluxe Aluminum Transport Chair With Ha...        126.620870   
Sanitary Pads with Adhesive & Wings                        42.320000   
2-Ply Tissue/Poly Professional Towels, White, N...         25.270000   

                                                    Total_Amount  
                                                                  
Order_Items.product_name                                          
Disposable Emergency Blanket, Gray, Not Applicable  10556.050000  
Medline MoliCare Super Plus Disposable Adult Di...   8980.701887  
Medline Deluxe Aluminum Transport Chair With Ha...   5951.180870  
Sanitary Pads with Adhesive & Wings                  5628.560000  
2-Ply Tissue/Poly Professional Towels, White, N...   5559.400000

In [23]:
the_most_popular("Order_Items.product_name","sum")

Order_Items.qty       \
                                                              size  sum   
Order_Items.product_name                                                  
TruTemp                                                          1  500   
Disposable Emergency Blanket, Gray, Not Applicable              14  305   
2-Ply Tissue/Poly Professional Towels, White, N...               2  220   
Medline MoliCare Super Plus Disposable Adult Di...              53  164   
Sanitary Pads with Adhesive & Wings                              2  133   

                                                                \
                                                          mean   
Order_Items.product_name                                         
TruTemp                                             500.000000   
Disposable Emergency Blanket, Gray, Not Applicable   21.785714   
2-Ply Tissue/Poly Professional Towels, White, N...  110.000000   
Medline MoliCare Super Plus Disposable Adult Di...    3.094340   
Sanitary Pads with Adhesive & Wings                  66.500000   

                                                   Order_Items.price  \
                                                                mean   
Order_Items.product_name                                               
TruTemp                                                     2.990000   
Disposable Emergency Blanket, Gray, Not Applicable         34.610000   
2-Ply Tissue/Poly Professional Towels, White, N...         25.270000   
Medline MoliCare Super Plus Disposable Adult Di...         54.760377   
Sanitary Pads with Adhesive & Wings                        42.320000   

                                                    Total_Amount  
                                                                  
Order_Items.product_name                                          
TruTemp                                              1495.000000  
Disposable Emergency Blanket, Gray, Not Applicable  10556.050000  
2-Ply Tissue/Poly Professional Towels, White, N...   5559.400000  
Medline MoliCare Super Plus Disposable Adult Di...   8980.701887  
Sanitary Pads with Adhesive & Wings                  5628.560000

In [24]:
the_most_popular("Customers.company","total")

Order_Items.qty                 Order_Items.price  \
                             size  sum       mean              mean   
Customers.company                                                     
Company59                      15  381  25.400000         34.610000   
Company343                      2  133  66.500000         42.320000   
Company17                       2   27  13.500000        177.190000   
Company159                     11   47   4.272727         80.058182   
Company281                      4   43  10.750000         62.910000   

                   Total_Amount  
                                 
Customers.company                
Company59          13186.410000  
Company343          5628.560000  
Company17           4784.130000  
Company159          3762.734545  
Company281          2705.130000

In [25]:
the_most_popular("Customers.company","sum")

Order_Items.qty                 Order_Items.price  \
                             size  sum       mean              mean   
Customers.company                                                     
Company59                      15  381  25.400000         34.610000   
Company343                      2  133  66.500000         42.320000   
Company86                      11  105   9.545455         18.400000   
Company130                     17   71   4.176471         28.382353   
Company171                     12   47   3.916667         31.941667   

                   Total_Amount  
                                 
Customers.company                
Company59          13186.410000  
Company343          5628.560000  
Company86           1932.000000  
Company130          2015.147059  
Company171          1501.258333

In [26]:
the_most_popular("Full_Name","total")

Order_Items.qty             Order_Items.price  \
                                      size  sum   mean              mean   
Full_Name                                                                  
United promotion promotion               2  505  252.5            68.425   
RoseAnn Reyes                           10  300   30.0            34.610   
Burten  Leibowitz                        2  133   66.5            42.320   
David Oconnell                           2  220  110.0            25.270   
Lalith Paulus                            2   27   13.5           177.190   

                           Total_Amount  
                                         
Full_Name                                
United promotion promotion    34554.625  
RoseAnn Reyes                 10383.000  
Burten  Leibowitz              5628.560  
David Oconnell                 5559.400  
Lalith Paulus                  4784.130

In [27]:
the_most_popular("Full_Name","sum")

Order_Items.qty                  Order_Items.price  \
                                      size  sum        mean              mean   
Full_Name                                                                       
United promotion promotion               2  505  252.500000            68.425   
RoseAnn Reyes                           10  300   30.000000            34.610   
David Oconnell                           2  220  110.000000            25.270   
Burten  Leibowitz                        2  133   66.500000            42.320   
Eleni Ventsanos                         11  105    9.545455            18.400   

                           Total_Amount  
                                         
Full_Name                                
United promotion promotion    34554.625  
RoseAnn Reyes                 10383.000  
David Oconnell                 5559.400  
Burten  Leibowitz              5628.560  
Eleni Ventsanos                1932.000

In [28]:
the_most_popular("Products.shopping_brand","total")

Order_Items.qty                  \
                                        size   sum      mean   
Products.shopping_brand                                        
MEDLINE                                 2672  4939  1.848428   
Drive Medical                            535   567  1.059813   
HARTMANN-CONCO INC                       122   324  2.655738   
Wenzelite                                 36    48  1.333333   
Fabrication Enterprises Inc.              85   112  1.317647   

                             Order_Items.price   Total_Amount  
                                          mean                 
Products.shopping_brand                                        
MEDLINE                              59.674308  294731.405408  
Drive Medical                       120.378224   68254.453178  
HARTMANN-CONCO INC                   58.843525   19065.301967  
Wenzelite                           179.534722    8617.666667  
Fabrication Enterprises Inc.         49.979882    5597.746824

In [29]:
the_most_popular("Products.shopping_brand","sum")

Order_Items.qty                  \
                                        size   sum      mean   
Products.shopping_brand                                        
MEDLINE                                 2672  4939  1.848428   
Drive Medical                            535   567  1.059813   
HARTMANN-CONCO INC                       122   324  2.655738   
CanDo&reg;                               113   182  1.610619   
Fabrication Enterprises Inc.              85   112  1.317647   

                             Order_Items.price   Total_Amount  
                                          mean                 
Products.shopping_brand                                        
MEDLINE                              59.674308  294731.405408  
Drive Medical                       120.378224   68254.453178  
HARTMANN-CONCO INC                   58.843525   19065.301967  
CanDo&reg;                           30.438761    5539.854513  
Fabrication Enterprises Inc.         49.979882    5597.746824

In [30]:
the_most_popular("Orders.payment_method","total")

Order_Items.qty                Order_Items.price  \
                                 size  sum      mean              mean   
Orders.payment_method                                                    
Credit Card                       373  525  1.407507         86.151340   
PayPal                            318  510  1.603774         49.338113   
Check                              29   66  2.275862         81.778276   
Cash                                2    3  1.500000        115.980000   

                       Total_Amount  
                                     
Orders.payment_method                
Credit Card            45229.453753  
PayPal                 25162.437736  
Check                   5397.366207  
Cash                     347.940000

In [31]:
the_most_popular("Orders.shipping_method","sum")

Order_Items.qty                   Order_Items.price  \
                                    size   sum        mean              mean   
Orders.shipping_method                                                         
11|Ground                           2474  4446    1.797090         62.332078   
0|Standard Shipping                  902  1188    1.317073         63.942938   
-1|Free Shipping                     267  1085    4.063670        125.706742   
SOUTHWESTERN MOTOR TRNAS               1   200  200.000000         25.270000   
0|Free Shipping                       91    97    1.065934        142.716044   

                           Total_Amount  
                                         
Orders.shipping_method                   
11|Ground                 277128.417041  
0|Standard Shipping        75964.210244  
-1|Free Shipping          136391.814607  
SOUTHWESTERN MOTOR TRNAS    5054.000000  
0|Free Shipping            13843.456264

In [32]:
# Build a recommendation system using a matrix
product_recommend=df.pivot_table(index="Orders.customer_id",
                                 columns='Order_Items.product_name',
                                 values='Order_Items.qty').fillna(0)
product_recommend

In [33]:
the_most_popular("Order_Items.product_name","sum")

Order_Items.qty       \
                                                              size  sum   
Order_Items.product_name                                                  
TruTemp                                                          1  500   
Disposable Emergency Blanket, Gray, Not Applicable              14  305   
2-Ply Tissue/Poly Professional Towels, White, N...               2  220   
Medline MoliCare Super Plus Disposable Adult Di...              53  164   
Sanitary Pads with Adhesive & Wings                              2  133   

                                                                \
                                                          mean   
Order_Items.product_name                                         
TruTemp                                             500.000000   
Disposable Emergency Blanket, Gray, Not Applicable   21.785714   
2-Ply Tissue/Poly Professional Towels, White, N...  110.000000   
Medline MoliCare Super Plus Disposable Adult Di...    3.094340   
Sanitary Pads with Adhesive & Wings                  66.500000   

                                                   Order_Items.price  \
                                                                mean   
Order_Items.product_name                                               
TruTemp                                                     2.990000   
Disposable Emergency Blanket, Gray, Not Applicable         34.610000   
2-Ply Tissue/Poly Professional Towels, White, N...         25.270000   
Medline MoliCare Super Plus Disposable Adult Di...         54.760377   
Sanitary Pads with Adhesive & Wings                        42.320000   

                                                    Total_Amount  
                                                                  
Order_Items.product_name                                          
TruTemp                                              1495.000000  
Disposable Emergency Blanket, Gray, Not Applicable  10556.050000  
2-Ply Tissue/Poly Professional Towels, White, N...   5559.400000  
Medline MoliCare Super Plus Disposable Adult Di...   8980.701887  
Sanitary Pads with Adhesive & Wings                  5628.560000

In [34]:
product_TruTemp=product_recommend["TruTemp"]
product_TruTemp.head()

Orders.customer_id
3    0.0
4    0.0
5    0.0
7    0.0
8    0.0
Name: TruTemp, dtype: float64

In [35]:
product_TruTemp.value_counts()

0.0      3053
500.0       1
Name: TruTemp, dtype: int64

In [36]:
product_recommend[["TruTemp","Disposable Emergency Blanket, Gray, Not Applicable"]].corr()

Order_Items.product_name,TruTemp,"Disposable Emergency Blanket, Gray, Not Applicable"
Order_Items.product_name,,
TruTemp,1.000000,-0.000545
"Disposable Emergency Blanket, Gray, Not Applicable",-0.000545,1.000000


In [44]:
truTemp_df=pd.DataFrame()
truTemp_df["Product"]=product_recommend.corrwith(product_TruTemp).index
truTemp_df["Correlasion"]=product_recommend.corrwith(product_TruTemp).values
truTemp_df.sort_values(("Correlasion"),ascending=False)

,Product,Correlasion
1675,TruTemp,1.000000
1197,"Phlebotomy Sharps Containers, Red, 1.000 QT",0.707028
249,"CURAD Stretch Vinyl Exam Gloves, Medium",-0.000328
1022,Medline MSC327100 Capri Bladder Control Pads,-0.000328
15,"2-Ply Tissue/Poly Professional Towels, White, ...",-0.000328
1610,Suresite Window Transparent Dressings,-0.000328
648,"FitRight Active Male Guards, 6"" X 11""",-0.000328
1273,Protection Plus Overnight Protective Underwear...,-0.000328
1260,"Protection Plus Classic Protective Underwear, ...",-0.000328
315,CanDo&reg; Latex Free Exercise Band - 50 yard ...,-0.000328


In [39]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
sparse_matrix=vect.fit_transform(df)

In [40]:
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=vect.get_feature_names())
df.sample(10)

,company,create_date,currency,customer_id,customers,fname,full_name,id,last_modified,lname,order_items,order_number,orders,parent,payment_method,placed_date,price,product_name,products,qty,shipping_method,shopping_brand,status,subtotal,total
14,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
10,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [41]:
df.shape

(25, 25)

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
pd.DataFrame(cosine_similarity(df,df))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,1.0,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
1,0.5,1.0,0.5,0.5,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
2,0.5,0.5,1.0,0.5,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
3,0.5,0.5,0.5,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
4,0.5,0.5,0.5,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
5,0.5,0.0,0.0,0.0,0.0,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5
6,0.0,0.0,0.0,0.0,0.0,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5
7,0.0,0.5,0.0,0.0,0.0,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5
8,0.0,0.0,0.5,0.0,0.0,0.5,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5
9,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5
